https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1


https://towardsdatascience.com/similarity-metrics-in-nlp-acc0777e234c 

https://www.youtube.com/watch?v=Ey81KfQ3PQU


Model semantic search: 'multi-qa-MiniLM-L6-cos-v1'

Model assymetric : 'msmarco-MiniLM-L-6-v3'

In [1]:
import pandas as pd
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.Logistic_regression import *
from Functions.Models.evaluation import *
from Data_with_NLP import *
import pandas as pd
# import pandasql as ps
import matplotlib.pyplot as plt
# Import argsort
from numpy import argsort
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.Logistic_regression import *
from Functions.Models.evaluation import *
from Data_with_NLP import *
from Functions.pipeline import *

In [2]:
protocols = pd.read_csv('/Users/gabrielabib/Desktop/NOVA_SBE/PBL/BERT/protocols.csv')
protocols_acceptance = protocols[['Acceptance']]
protocols_refusal = protocols[['Refusal']]
protocols_acceptance.dropna(inplace=True)

/var/folders/g5/43zxmsfs73v6lgj3_2k04qj80000gp/T/ipykernel_2111/106944270.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protocols_acceptance.dropna(inplace=True)


In [3]:
# # 1. Data cleaning
lower_text(protocols_acceptance,'Acceptance', 'Acceptance')
lower_text(protocols_refusal,'Refusal', 'Refusal')
remove_stop_words(protocols_acceptance,'Acceptance', 'Acceptance')
remove_stop_words(protocols_refusal,'Refusal', 'Refusal')
spacy_lemmatizer(protocols_acceptance,'Acceptance', 'Acceptance')
spacy_lemmatizer(protocols_refusal,'Refusal', 'Refusal')

protocols_acceptance

/Users/gabrielabib/Desktop/NOVA_SBE/PBL/PBL-HGO/Functions/NLP/alertp1_nlp.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column] = df[column].str.lower()
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gabrielabib/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gabrielabib/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/gabrielabib/Desktop/NOVA_SBE/PBL/PBL-HGO/Functions/NLP/alertp1_nlp.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,Acceptance
0,cefaleia crónico refratária associar abuso med...
1,distúrbio movimento distúrbio equilíbrio quer ...
2,doença neuromuscular genético dor neuropático ...
3,distúrbio aprendizagem retardor psicomotor
4,síndromes demencial demência primário secundár...
5,esclerose múltiplo outro doença desmielinizant...


In [4]:
acceptance_list = protocols_acceptance['Acceptance'].tolist()
acceptance_list

['cefaleia crónico refratária associar abuso medicamentoso',
 'distúrbio movimento distúrbio equilíbrio quer frequente distonia doença parkinson parkinsonismo esclarecer tremor essencial difícil controle ataxias esclarecer coreia / balismos etc',
 'doença neuromuscular genético dor neuropático refratário parestesia recorrente e/ou progressivo paraplegia paresio esclerose lateral amiotrófico suspeito miopatia miastenia grave outro transtorno neuromuscular amiotrofia espinhal síndromes correlato polineuropatia outro transtorno sistema nervoso periférico snp doenço neurogenético',
 'distúrbio aprendizagem retardor psicomotor',
 'síndromes demencial demência primário secundário dúvida diagnósticar demência rapidamente progressivo',
 'esclerose múltiplo outro doença desmielinizante neuroimunológico']

In [5]:
[acceptance_list[0]]

['cefaleia crónico refratária associar abuso medicamentoso']

In [6]:
alertP1 = pd.read_csv('/Users/gabrielabib/Desktop/NOVA_SBE/PBL/PBL-HGO/alertp1_original.csv')
alertP1 = alertP1[alertP1['text_length']>0]


In [7]:
alertP1_simplified = alertP1[['COD_REFERENCIA', 'Texto']]

In [8]:
alertP1_simplified.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1773 entries, 0 to 1781
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   COD_REFERENCIA  1773 non-null   object
 1   Texto           1773 non-null   object
dtypes: object(2)
memory usage: 41.6+ KB


In [9]:
lower_text(alertP1_simplified,'Texto', 'Texto')
remove_stop_words(alertP1_simplified,'Texto', 'Texto_Stop_Words')
spacy_lemmatizer(alertP1_simplified,'Texto_Stop_Words', 'text_lemmatized')
alertP1_simplified

/Users/gabrielabib/Desktop/NOVA_SBE/PBL/PBL-HGO/Functions/NLP/alertp1_nlp.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column] = df[column].str.lower()
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gabrielabib/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gabrielabib/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/gabrielabib/Desktop/NOVA_SBE/PBL/PBL-HGO/Functions/NLP/alertp1_nlp.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,COD_REFERENCIA,Texto,Texto_Stop_Words,text_lemmatized
0,LQHSOQLH,"utente de 75 anos, refere tremor desde há 5 an...",75 anos tremor desde 5 anos esquecimentos difi...,75 ano tremor desde 5 ano esquecimento dificul...
1,LOHLTRTS,avaliação neurologica para avaliação da toma d...,avaliação neurologica avaliação toma anti-epil...,avaliação neurologico avaliação tomar anti-epi...
2,LQHROQHL,cefaleias,cefaleias,cefaleia
3,LPHQULSU,"utente do sexo feminino com 30 anos de idade, ...",sexo feminino 30 anos idade aparentemente said...,sexo feminino 30 ano idade aparentemente saidá...
4,LPHHVHVL,sem relatorio clinico,relatorio clinico,relatorio clinico
...,...,...,...,...
1777,LRHSSOHP,estenose carotidea assintomática bilateral < 7...,estenose carotidea assintomática bilateral < 7...,estenose carotidea assintomático bilateral < 7...
1778,LRHLHOOH,61 anos operada por patologia benigna da mama...,61 anos operada patologia benigna mama direita...,61 ano operar patologia benigno mama direito a...
1779,LRHOVHOP,utente de 69a com perda de conhecimento a 6.5....,69a perda conhecimento 6.5.2015 elipotomia tce...,69a perda conhecimento 6.5.2015 elipotomia tce...
1780,LRHRTLPV,utente de 70a com historia de lipotimia em mai...,70a historia lipotimia maio 2015. eeg-29.5.201...,70a historia lipotimer maio 2015 . eeg-29.5. 2...


In [10]:
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import torch


/usr/local/Caskroom/miniconda/base/envs/pbl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

In [12]:
# # THIS SHOULD BE DELETED

# alertP1_simplified = alertP1_simplified[:100]
# alertP1_simplified

In [13]:
# referrals = alertP1_simplified['text_lemmatized'].tolist()


# for index, protocol in enumerate(acceptance_list):

#     sentences = [protocol] + referrals

#     # the following line is only necessary if you want to limit the number of sentences. MUST BE COMMENTED OUT TO RUN ON FULL DATASET
#     # sentences = sentences[:100]

#     # initialize dictionary to store tokenized sentences
#     tokens = {'input_ids': [], 'attention_mask': []}

#     for sentence in sentences:
#         # encode each sentence and append to dictionary
#         new_tokens = tokenizer.encode_plus(sentence, max_length=128,
#                                         truncation=True, padding='max_length',
#                                         return_tensors='pt')
#         tokens['input_ids'].append(new_tokens['input_ids'][0])
#         tokens['attention_mask'].append(new_tokens['attention_mask'][0])

#     # reformat list of tensors into single tensor
#     tokens['input_ids'] = torch.stack(tokens['input_ids'])
#     tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
        
#     outputs = model(**tokens)
#     outputs.keys()

#     embeddings = outputs.last_hidden_state
#     # embeddings

#     attention_mask = tokens['attention_mask']
#     # attention_mask.shape
#     mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
#     # mask.shape

#     masked_embeddings = embeddings * mask
#     # masked_embeddings.shape

#     summed = torch.sum(masked_embeddings, 1)
#     # summed.shape

#     summed_mask = torch.clamp(mask.sum(1), min=1e-9)
#     summed_mask.shape

#     mean_pooled = summed / summed_mask

#     # convert from PyTorch tensor to numpy array
#     mean_pooled = mean_pooled.detach().numpy()

#     # calculate
#     results = cosine_similarity(
#         [mean_pooled[0]],
#         mean_pooled[1:]
#     )

#     alertP1_simplified[index] = results[0]



In [14]:
referrals = alertP1_simplified['text_lemmatized'].tolist()


protocol = acceptance_list[0]

sentences = [protocol] + referrals

# the following line is only necessary if you want to limit the number of sentences. MUST BE COMMENTED OUT TO RUN ON FULL DATASET
# sentences = sentences[:100]

# initialize dictionary to store tokenized sentences
tokens = {'input_ids': [], 'attention_mask': []}

for sentence in sentences:
    # encode each sentence and append to dictionary
    new_tokens = tokenizer.encode_plus(sentence, max_length=128,
                                    truncation=True, padding='max_length',
                                    return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

# reformat list of tensors into single tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
    
outputs = model(**tokens)
outputs.keys()

embeddings = outputs.last_hidden_state
# embeddings

attention_mask = tokens['attention_mask']
# attention_mask.shape
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
# mask.shape

masked_embeddings = embeddings * mask
# masked_embeddings.shape

summed = torch.sum(masked_embeddings, 1)
# summed.shape

summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape

mean_pooled = summed / summed_mask

# convert from PyTorch tensor to numpy array
mean_pooled = mean_pooled.detach().numpy()

# calculate
results = cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

alertP1_simplified['0'] = results[0]




: 

: 

In [ ]:
sentences

['cefaleia crónico refratária associar abuso medicamentoso',
 '75 ano tremor desde 5 ano esquecimento dificuldade reter informação momento queda frequente hta diabete último análise -2/07/14 hb A1c-7,7 hdl ct-54 ct-193 tg-176 hb-12,2 creat-1,2 microalbuminúria-7,3 creat-1,2 fazer tac ce-1/09/14 acentuação ventriculos cerebral leucoencefalopatia micro-angiopático crónico acentuação sulco fronto-parietal sobretudo esq ligeiro hipodensidade periférico cerebeloso calcificação foice inter-hemisférico porção sup-anterior medicar diamicron metformina+sitagliptina 850 + 50 2xdia irbesartan+hctz 300 + 12,5 1em jejum omeprazol-20 mg sinvastatina 40 mg tromalyt 150,1cp almoço',
 'avaliação neurologico avaliação tomar anti-epilepticos reajustamento terapeutico doente nao notar internamento 2007 enfarte isquémico cerebelosoe vermiani picateve ataxio marcha 2007 ficar medicar lamotriginar 100- 1 + 1 tryptizolr agora ligeiro desiquilibrios exame neurologico sumario parecer completamente normal',
 'ce